In [1]:
import os
os.chdir("../..")

In [2]:
import torch
from utils.dataset_loader import *
from utils.model_utils import *
from utils.train_utils import *
from utils.metrics import *
from utils.visualization import *

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"[INFO] Using device: {device}")

[INFO] Using device: cuda


In [4]:
data_dir = "wildfire_dataset_scaled"
batch_size = 32
num_epochs = 10
learning_rate = 0.001
num_classes = 2

In [5]:
print("[INFO] Loading datasets...")
train_loader, val_loader, test_loader = load_datasets(data_dir, batch_size, augmentation="baseline")
print("[INFO] Datasets loaded successfully!")



2024-12-21 22:39:37,323 - INFO - Loading datasets from wildfire_dataset_scaled with augmentation type 'baseline'.
2024-12-21 22:39:37,324 - INFO - Applying baseline augmentations: Resize and Normalize.
2024-12-21 22:39:37,332 - INFO - Initialized AlbumentationsDataset with root: wildfire_dataset_scaled/train
2024-12-21 22:39:37,335 - INFO - Initialized AlbumentationsDataset with root: wildfire_dataset_scaled/val
2024-12-21 22:39:37,336 - INFO - Initialized AlbumentationsDataset with root: wildfire_dataset_scaled/test
2024-12-21 22:39:37,337 - INFO - Datasets initialized. Preparing DataLoaders...
2024-12-21 22:39:37,337 - INFO - Using WeightedRandomSampler for class balancing.
2024-12-21 22:39:37,338 - INFO - Computing class weights from directory: wildfire_dataset_scaled/train
2024-12-21 22:39:37,339 - INFO - Class 'fire' has 730 samples.
2024-12-21 22:39:37,340 - INFO - Class 'nofire' has 1157 samples.
2024-12-21 22:39:37,341 - INFO - Computed class weights: {'fire': 2.584931506849315

[INFO] Loading datasets...


2024-12-21 22:39:38,565 - INFO - DataLoaders created successfully.


[INFO] Datasets loaded successfully!


In [6]:
print("\n[INFO] Starting Baseline Training for Model: mobilenetv2\n")
print("[INFO] Initializing mobilenetv2 model...")
model = initialize_model(
    model_name="mobilenet_v2",
    num_classes=num_classes,
    pretrained=True,
    freeze_all=True  # Baseline: Freeze all layers
)
print("[INFO] Model initialized successfully!")


[INFO] Starting Baseline Training for Model: mobilenetv2

[INFO] Initializing mobilenetv2 model...
[INFO] Model initialized successfully!


In [7]:
print("[INFO] Setting up loss function and optimizer...")
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=learning_rate)
print("[INFO] Loss function and optimizer set up!")

[INFO] Setting up loss function and optimizer...
[INFO] Loss function and optimizer set up!


In [8]:
print("[INFO] Starting model training...")
history = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=None,  # No learning rate scheduler for baseline
    device=device,
    save_path=f"outputs/models/baseline/mobilenetv2.pt",
    early_stop_patience=8,
    monitor_metric="val_loss",
)
print("[INFO] Training completed successfully!")

[INFO] Starting model training...

Starting training...



[INFO] Best model saved with val_loss: 0.4711
Epoch [1]: Train Loss: 0.5474, Train Acc: 0.7493 | Val Loss: 0.4711, Val Acc: 0.8159, Val Recall: 0.7967, Val F1: 0.8412


KeyboardInterrupt: 

In [ ]:
print("[INFO] Evaluating the model...")
metrics = evaluate_model(
    model=model,
    test_loader=test_loader,
    classes=["No Fire", "Fire"],
    device=device
)
print(f"\n[INFO] Metrics for ResNet18:\n{metrics}")

In [ ]:
# Training and Validation Curves
plot_training(history, f"outputs/efficientnet_baseline_training_curve.png")

# Confusion Matrix
plot_confusion_matrix(
    cm=metrics["confusion_matrix"],
    classes=["No Fire", "Fire"],
    output_path=f"outputs/efficientnet_baseline_confusion_matrix.png"
)

# Precision-Recall Curve
if "y_probs" in metrics:
    y_true = metrics["y_true"]
    y_probs = [prob[1] for prob in metrics["y_probs"]]
    plot_precision_recall(
        y_true=y_true,
        y_scores=y_probs,
        output_path=f"outputs/efficientnet_baseline_precision_recall_curve.png"
    )

    # ROC Curve
    plot_roc_curve(
        y_true=y_true,
        y_scores=y_probs,
        output_path=f"outputs/efficientnet_baseline_roc_curve.png"
    )

print("[INFO] All results saved successfully!")